# Cleaning data' 2: Electric Boogaloo

In the previous blog post (https://github.com/GTouzin/Portfolio/blob/master/R/IMDB_data_cleaning.ipynb), I started to clean the data scraped from IMDB by Kaggle user chuansun76 (https://www.kaggle.com/deepmatrix). I wanted to find the missing data from the variables "budget" and "gross" by using a Python script to scrape the data from the web. Since chuansun76 was kind enough to share his source code with the IMDB data set, I have modified his code to scrape the data from the site www.the-numbers.com. You can find the details of that scrapping process here https://github.com/GTouzin/Portfolio/tree/master/Python (full notebook coming soon).

In this post, I'll clean the new data and merge it with the data from the original data set.

In [59]:
library(dplyr)
library(data.table)
library(bit64)
library(plotly)
options(scipen=999)

# Merging the data 

I saved the IMDB data set partially clean in the file "movies_clean_na.csv" and data scrape from the site the-number.com in the file "scrap_gross_budget.csv". So the first thing I do is loading those two data sets and look at the structure of the table.

In [60]:
movies <- read.csv("movies_clean_na2.csv",sep=";",stringsAsFactors = F)
setDT(movies)
head(movies)
budget <- read.csv ("scrap_gross_budget_copie.csv",stringsAsFactors = F)
setDT(budget)
head(budget)

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genres_News,genres_Reality.TV,genres_Romance,genres_Sci.Fi,genres_Short,genres_Sport,genres_Thriller,genres_War,genres_Western,movie_facebook_likes.1
Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,0,0,0,1,0,0,0,0,0,33000
Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,0,0,0,0,0,0,0,0,0,0
Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,0,0,0,0,0,0,1,0,0,85000
Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,0,0,0,0,0,0,1,0,0,164000
Color,Doug Walker,NA,136,131,NA,Rob Walker,131,NA,Documentary,...,0,0,0,0,0,0,0,0,0,0
Color,Andrew Stanton,462,132,475,530,Samantha Morton,640,73058679,Action|Adventure|Sci-Fi,...,0,0,0,1,0,0,0,0,0,24000


title_year,budget,movie_title,gross
2009,425000000,Avatar,760507625
2015,306000000,Star Wars Ep. VII: The Force Awakens,936662225
2007,300000000,Pirates of the Caribbean: At Worlds End,309420425
2015,300000000,Spectre,200074175
2012,275000000,The Dark Knight Rises,448139099
2013,275000000,The Lone Ranger,89302115


In [61]:
str(budget)

Classes 'data.table' and 'data.frame':	5241 obs. of  4 variables:
 $ title_year : chr  "2009" "2015" "2007" "2015" ...
 $ budget     : chr  "425000000" "306000000" "300000000" "300000000" ...
 $ movie_title: chr  "Avatar" "Star Wars Ep. VII: The Force Awakens" "Pirates of the Caribbean: At Worlds End" "Spectre" ...
 $ gross      : chr  "760507625" "936662225" "309420425" "200074175" ...
 - attr(*, ".internal.selfref")=<externalptr> 


We see that the variable "budget" in the new data set is encoded as a chr variable, so I change it to a numerical variable.

In [62]:
budget$budget<-as.numeric(budget$budget)
str(budget)

Warning message in eval(expr, envir, enclos):
"NAs introduits lors de la conversion automatique"

Classes 'data.table' and 'data.frame':	5241 obs. of  4 variables:
 $ title_year : chr  "2009" "2015" "2007" "2015" ...
 $ budget     : num  425000000 306000000 300000000 300000000 275000000 275000000 275000000 260000000 258000000 250000000 ...
 $ movie_title: chr  "Avatar" "Star Wars Ep. VII: The Force Awakens" "Pirates of the Caribbean: At Worlds End" "Spectre" ...
 $ gross      : chr  "760507625" "936662225" "309420425" "200074175" ...
 - attr(*, ".internal.selfref")=<externalptr> 


The function below count the number of missing values in the column "gross" and "budget" of the original data set. Those are the values that I want to fill in with the new values.

In [63]:
print(paste0("Number of missing values in budget: ",sum(is.na(movies[, budget]))))
print(paste0("Number of missing values in gross: ",sum(is.na(movies[, gross]))))

[1] "Number of missing values in budget: 395"
[1] "Number of missing values in gross: 773"


I have to make a custom function to fill the missing "budget" and "gross" values with the new values. The function below return a vector who's a copy of the column with the missing data. To do this, the function makes a copy of this column, it compares the variables "movie_title" and "title_year" of a movie in the initial table with those of the movie in the table containing the scrape data with the intention to find the corresponding movie, then verify if the value is missing in the copied column and, if that's the case, it adds the new value. 

In [64]:
fill_budget<-function(title_ref,year_ref,budget_ref,title_to_fill,year_to_fill,budget_to_fill){
    value<-budget_to_fill
    for (i in 1:length(title_to_fill))
    {
       index<-which(gsub(" ", "", tolower(title_ref), fixed = TRUE) %in% gsub(" ", "", tolower(title_to_fill[i]), fixed = TRUE)
                    & gsub(" ", "", tolower(year_ref), fixed = TRUE) %in% gsub(" ", "", tolower(year_to_fill[i]), fixed = TRUE))
       
        if(length(index)==0||length(index)>=2)
        {
           #print(paste0(title_to_fill[i],": ",length(index)))            
        }
        else
        {
            if(is.na(value[i]))
            {
                value[i]<-budget_ref[index[1]]
                #print(paste0(title_to_fill[i],": ",budget_ref[index[1]]))  
            }  
        }
    } 
    return(value)
}

First, I used my function on the "budget" variable.

In [65]:
temp<-fill_budget(budget$movie_title,budget$title_year,budget$budget,movies$movie_title,movies$title_year,movies$budget)

I count the numbers of missing observations that have been replaced by my function.

In [66]:
print(paste0("Number of NA in the new vector: ",sum(is.na(temp))))
print(paste0("Number of NA in the original vector: ",sum(is.na(movies[, budget]))))
print(paste0("Ratio: ",sum(is.na(temp))/sum(is.na(movies[, budget]))))


[1] "Number of NA in the new vector: 229"
[1] "Number of NA in the original vector: 395"
[1] "Ratio: 0.579746835443038"


I filled more than half of the missing data: not bad! By looking at the name of the movies who still have a missing value, I realized that some movies names in the reference data set are a little bit different than the movie name in the data set with the new budget values. Generally, the difference is small, often a quote missing is the cause of the difference. When I'll have more time, I would like to write a script to fix that problem.

Let's make sure my function didn't create any error in the data by comparing the value of the original vector with the vector I created.

In [67]:
print(paste0("Number of missing values in movies: ",sum(is.na(movies[, budget]))))
print(paste0("Number of missing values in the new vector: ",sum(is.na(temp))))
print(paste0("Number of difference between the two vector: ",sum(movies[, budget]!=temp, na.rm=TRUE)))

[1] "Number of missing values in movies: 395"
[1] "Number of missing values in the new vector: 229"
[1] "Number of difference between the two vector: 0"


Since the only difference between the two vectors are present on the rows where there's a missing value in the first vector, my function didn't change a original value in the first vector. Reassured by that fact, I copied the vector returned by my function in the original data set.

In [68]:
movies$budget<-temp

I repeat those steps to fill the missing "gross" values.

In [69]:
temp<-fill_budget(budget$movie_title,budget$title_year,budget$gross,movies$movie_title,movies$title_year,movies$gross)

In [70]:
print(paste0("Number of NA in the new vector: ",sum(is.na(temp))))
print(paste0("Number of NA in the original vector: ",sum(is.na(movies[, gross]))))
print(paste0("Ratio: ",sum(is.na(temp))/sum(is.na(movies[, gross]))))

[1] "Number of NA in the new vector: 390"
[1] "Number of NA in the original vector: 773"
[1] "Ratio: 0.504527813712807"


In [71]:
print(paste0("Number of missing values in movies: ",sum(is.na(movies[, gross]))))
print(paste0("Number of missing values in the new vector: ",sum(is.na(temp))))
print(paste0("Number of difference between the two vector: ",sum(movies[, gross]!=temp, na.rm=TRUE)))

[1] "Number of missing values in movies: 773"
[1] "Number of missing values in the new vector: 390"
[1] "Number of difference between the two vector: 0"


In [72]:
movies$gross<-temp

Here's the numbers of missing values in the data set for each variable. Since most of the variables are categorical with too much level to allow for two observations to be similar I'm afraid that to impute missing data would generate too much noise in the data. So, I will capitalise on the fact that my data set is large and I won't use those observations while creating my model. Also, I won't delete the observations with missing values, since the information in those observations will be useful in the estimation of the distribution for each variable.  

In [73]:
sapply(movies, function(y) sum(length(which(is.na(y)))))

color             director_name    num_critic_for_reviews 
                        0                         1                        43 
                 duration   director_facebook_likes    actor_3_facebook_likes 
                        0                         4                        20 
             actor_2_name    actor_1_facebook_likes                     gross 
                        0                         8                       390 
                   genres              actor_1_name               movie_title 
                        0                         0                         0 
          num_voted_users cast_total_facebook_likes              actor_3_name 
                        0                         1                         0 
     facenumber_in_poster             plot_keywords           movie_imdb_link 
                       14                         0                         1 
     num_user_for_reviews                  language                   country 
                       19                         0                         0 
           content_rating                    budget                title_year 
                        1                       229                         7 
   actor_2_facebook_likes                imdb_score              aspect_ratio 
                       10                         1                       307 
     movie_facebook_likes             genres_Action          genres_Adventure 
                        0                         0                        66 
         genres_Animation          genres_Biography             genres_Comedy 
                       66                        66                        66 
             genres_Crime        genres_Documentary              genres_Drama 
                       66                        66                        66 
            genres_Family            genres_Fantasy          genres_Film.Noir 
                       66                        66                        66 
         genres_Game.Show            genres_History             genres_Horror 
                       66                        66                        66 
             genres_Music            genres_Musical            genres_Mystery 
                       66                        66                        66 
              genres_News         genres_Reality.TV            genres_Romance 
                       66                        66                        66 
            genres_Sci.Fi              genres_Short              genres_Sport 
                       66                        66                        66 
          genres_Thriller                genres_War            genres_Western 
                       66                        66                        66 
   movie_facebook_likes.1 
                       66

In [74]:
#write.table(movies,file ="movies_clean_budget.csv",row.names=FALSE,sep=";")

# Converting the currency

Here I'm cheating a little bit: I realised there was a problem with the data way into the exploration phase, which will be my next article. But, even though I saw this problem way past this particular point, I believe that I should covert it while I'm cleaning the data. So without further adieu, let's look at the distribution of the "budget" variable.

In [75]:
budget_hist <- plot_ly(x=~movies$budget,type="histogram")
embed_notebook(budget_hist)

Warning message:
"Ignoring 229 observations"

In [76]:
budget_box<-plot_ly(y=~movies$budget,type="box")
embed_notebook(budget_box)

Warning message:
"Ignoring 229 observations"

When the outliers of your graph are so far away from the mode of the distribution that your histogram look like a density plot, something's wrong! Let's look at the table to have a better sense of what is happening. 

In [77]:
temp<-data.table(movie_title=movies$movie_title,budget=movies$budget,gross=movies$gross,
                 country=movies$country,title_year=movies$title_year)
temp<-temp[order(-budget)]
head(temp, n=10)

movie_title,budget,gross,country,title_year
The Host,12215500000,2201412,South Korea,2006
Lady Vengeance,4200000000,211667,South Korea,2005
Fateless,2500000000,195888,Hungary,2005
Princess Mononoke,2400000000,2298191,Japan,1997
Steamboy,2127519898,410388,Japan,2004
Akira,1100000000,439162,Japan,1988
Godzilla 2000,1000000000,10037390,Japan,1999
Kabhi Alvida Naa Kehna,700000000,3275443,India,2006
Tango,700000000,1687311,Spain,1998
Kites,600000000,1602466,India,2010


We see that foreing movie skew the distribution of that variable, especially the Asian films. That is because those "budget" and "gross" revenues are listed in a foreign currency and since most currencies have an exchange rate bigger than one with the US dollar, their value tends to skew negatively those two distributions.

I looked at the movie from South Korea to see how I could deal with this problem.

In [78]:
temp[temp$country=="South Korea"]

movie_title,budget,gross,country,title_year
The Host,12215500000,2201412,South Korea,2006
Lady Vengeance,4200000000,211667,South Korea,2005
Inchon,48000000,NA,South Korea,1981
Snowpiercer,39200000,4563029,South Korea,2013
Dragon Wars: D-War,35000000,10956379,South Korea,2007
The Last Godfather,13400000,163591,South Korea,2010
Tae Guk Gi: The Brotherhood of War,12800000,1110186,South Korea,2004
Operation Chromite,12620000,31662,South Korea,2016
Jungle Shuffle,10000000,NA,South Korea,2014
"\The Good, the Bad, the Weird \""""",10000000,128486,South Korea,2008


The budget for "Oldboy" is in US dollar. This observation tells me that I can't just convert all budgets of foreign movie without taking for consideration in which currency it is valued. Maybe I can convert the budget of foreign movies with budget high enough to indicate that it's not written in us dollar, let's say 50 000 000$? Of course, it's not an optimal approach, because some movies that cost under 50 million dollars will still be written in a foreign currency, but this approach would diminish the negative skew of the distribution and be the most time effective.

On the next table, the foreign movies with a budget of more than 50 million dollars are displayed in descending order.

In [79]:
head(temp[temp$country!="USA"&temp$budget>=50000000], n=20)

movie_title,budget,gross,country,title_year
The Host,12215500000,2201412,South Korea,2006
Lady Vengeance,4200000000,211667,South Korea,2005
Fateless,2500000000,195888,Hungary,2005
Princess Mononoke,2400000000,2298191,Japan,1997
Steamboy,2127519898,410388,Japan,2004
Akira,1100000000,439162,Japan,1988
Godzilla 2000,1000000000,10037390,Japan,1999
Kabhi Alvida Naa Kehna,700000000,3275443,India,2006
Tango,700000000,1687311,Spain,1998
Kites,600000000,1602466,India,2010


We see that there's quite a few American movies that are credited as foreign movie on IMDB (probably for tax reasons) but their budget are in US dollars. See, for exemple, King Kong, X-Men: The Last Stand and Harry Potter and the Half-Blood Prince.

So by using the strategy that I described above, not only some budget will still be written in a foreign currency, but I will change the value of some of the American movies for an incorrect value. I'll have to scrape some more data... 

# Getting the currency labels

By looking at the JSON file shared by Kaggle user chuansun76 containing the IMDB information, I saw that before the value for the "gross" and "budget" there's a prefix indicating in which currency the value is written. So by parcing that file, I was able to retrive this information and save it in the file find_estimated.csv.

In [80]:
currency <- fread("find_estimated.csv",stringsAsFactors = F)
setDT(currency)

In [81]:
head(currency)

title_year,currency,estimated,movie_title
2006,b'$',1,"b""""Pirates of the Caribbean: Dead Man's Chest"""""
2013,b'$',1,b'The Lone Ranger'
2013,b'$',1,b'Man of Steel'
2008,b'$',1,b'The Chronicles of Narnia: Prince Caspian'
2012,b'$',1,b'The Avengers'
2011,b'$',1,b'Pirates of the Caribbean: On Stranger Tides'


By looking at the table above, I realised I had a bit of data cleaning to do.

In [82]:
unique(currency$currency)

[1] "b'$'"   ""       "b''"   "b'£'"   "b'FRF'" "b'RUR'" "b'CNY'" "b'AUD'"
 [9] "b'HKD'" "b'CAD'" "b'JPY'" "b'NOK'" "b'DEM'" "b'THB'" "b'KRW'" "b'HUF'"
[17] "b'INR'" "b'DKK'" "b'CZK'" "b'ESP'" "b'NZD'" "b'CHF'" "b'BRL'" "b'ZAR'"
[25] "b'SEK'"

I dropped the first three characters in the variables "currency" and "movie_title".

In [83]:
currency$currency<-as.character(currency$currency)
currency$currency<-substr(currency$currency, 3, nchar(currency$currency)-1)
unique(currency$currency)

[1] "$"   ""    ""   "£"   "FRF" "RUR" "CNY" "AUD" "HKD" "CAD" "JPY" "NOK"
[13] "DEM" "THB" "KRW" "HUF" "INR" "DKK" "CZK" "ESP" "NZD" "CHF" "BRL" "ZAR"
[25] "SEK"

In [84]:
currency$movie_title<-as.character(currency$movie_title)
currency$movie_title<-substr(currency$movie_title, 3, nchar(currency$movie_title)-1)
sample(currency$movie_title, size=5)

[1] "Danny Collins"                  "The Brothers Solomon"          
[3] "House of Sand"                  "The Hammer"                    
[5] "\"It's a Wonderful Afterlife\""

Then, I have changed the characters '$', '€' and '£' for their country code and the empty string "" by NA.

In [85]:
currency$currency[currency$currency=='$']<-"USA"
currency$currency[currency$currency==currency$currency[currency$movie_title=="Asterix at the Olympic Games"]]<-"EU28"
currency$currency[currency$currency=='£']<-"GBR"
currency$currency[currency$currency=='']<-NA
unique(currency$currency)

[1] "USA"  NA     "EU28" "GBR"  "FRF"  "RUR"  "CNY"  "AUD"  "HKD"  "CAD" 
[11] "JPY"  "NOK"  "DEM"  "THB"  "KRW"  "HUF"  "INR"  "DKK"  "CZK"  "ESP" 
[21] "NZD"  "CHF"  "BRL"  "ZAR"  "SEK"

In the list above, the empty string "" is used as a currency, that tells me that some movies have no currency label, let's look at some of them.

In [86]:
head(currency$movie_title[is.na(currency$currency)])
length(currency$movie_title[is.na(currency$currency)])

[1] "The Good Dinosaur"                            
[2] "The Lovers"                                   
[3] "Godzilla Resurgence"                          
[4] "Harry Potter and the Deathly Hallows: Part II"
[5] "Godzilla Resurgence"                          
[6] "Harry Potter and the Deathly Hallows: Part I"

[1] 490

We see that the majority of those movies are American movies, which make sense since the U.S. dollar is the default currency, but some of them are just missing value. We also see that there's some duplicate in the table, for example:

In [87]:
subset(currency,movie_title=='Godzilla Resurgence')
str(currency)

title_year,currency,estimated,movie_title
2016,NA,NA,Godzilla Resurgence
2016,NA,NA,Godzilla Resurgence


Classes 'data.table' and 'data.frame':	5029 obs. of  4 variables:
 $ title_year : int  2006 2013 2013 2008 2012 2011 2012 2014 2012 2010 ...
 $ currency   : chr  "USA" "USA" "USA" "USA" ...
 $ estimated  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ movie_title: chr  "\"Pirates of the Caribbean: Dead Man's Chest\"" "The Lone Ranger" "Man of Steel" "The Chronicles of Narnia: Prince Caspian" ...
 - attr(*, ".internal.selfref")=<externalptr> 


Let's get rid of the duplicates.

In [88]:
setkey(currency,NULL)
currency<-unique(currency)
str(currency)

Classes 'data.table' and 'data.frame':	4907 obs. of  4 variables:
 $ title_year : int  2006 2013 2013 2008 2012 2011 2012 2014 2012 2010 ...
 $ currency   : chr  "USA" "USA" "USA" "USA" ...
 $ estimated  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ movie_title: chr  "\"Pirates of the Caribbean: Dead Man's Chest\"" "The Lone Ranger" "Man of Steel" "The Chronicles of Narnia: Prince Caspian" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [89]:
head(currency$movie_title[is.na(currency$currency)])

[1] "The Good Dinosaur"                            
[2] "The Lovers"                                   
[3] "Godzilla Resurgence"                          
[4] "Harry Potter and the Deathly Hallows: Part II"
[5] "Harry Potter and the Deathly Hallows: Part I" 
[6] "The A-Team            "

Now I'll use the data from the IMDB data.table to assign the USA label to American film with no currency data in the currency data.table.

In [90]:
change_US_currency<-function(x,y){
    value<-x$currency
    count<-0
    for (i in 1:length(x$currency)){
        
        index<-which(gsub(" ", "", tolower(y$movie_title), fixed = TRUE) %in%
                     gsub(" ", "", tolower(x$movie_title[i]), fixed = TRUE)
                    & gsub(" ", "", tolower(y$title_year), fixed = TRUE) %in%
                     gsub(" ", "", tolower(x$title_year[i]), fixed = TRUE))
       
        if(length(index)==0||length(index)>=2)
        {
           #print(paste0(title_to_fill[i],": ",length(index)))            
        }
        else
        {
            if(is.na(value[i]))
            {
                value[i]<-"USA"
                count=count+1 
            }  
        }

    }
    print(paste0("Nomber of change: ",count))
    return(value)
}

In [91]:
temp<-change_US_currency(currency,movies)

[1] "Nomber of change: 351"


Let's look at the number of Americain movies, foreign movies and missing data in the currency data.table and in the new vector to see if the function behaved correctly. If the function did his job, the number of foreign movies should be the same in the two data.table, the number of missing values should be lower in the new vector and the number of Americain movies in this table should be equal to the number of Americain movies in the first plus the difference in the number of missing values.

In [92]:
print(paste0("Number of foreign movies: ",length(currency$currency[currency$currency!="USA"])-sum(is.na(currency$currency))))
print(paste0("Number of american movies: ",nrow(currency[currency=="USA"])))
print(paste0("Number of missing values: ",sum(is.na(currency$currency))))
print(paste0("Sum of the number of american movies and missing values: ",
             nrow(currency[currency=="USA"])+sum(is.na(currency$currency))))
print(paste0("Sum of the number of american movies, foreign movies and missing values: ",
             nrow(currency[currency=="USA"])+sum(is.na(currency$currency))+sum(currency$currency!="USA",na.rm=TRUE)))
print(paste0("Total number of observation: ",length(currency$currency)))

[1] "Number of foreign movies: 216"
[1] "Number of american movies: 4208"
[1] "Number of missing values: 483"
[1] "Sum of the number of american movies and missing values: 4691"
[1] "Sum of the number of american movies, foreign movies and missing values: 4907"
[1] "Total number of observation: 4907"


In [93]:
print(paste0("Number of foreign movies: ",sum(temp!="USA",na.rm=TRUE)))
print(paste0("Number of american movies: ",sum(temp=="USA",na.rm=TRUE)))
print(paste0("Number of missing values: ",sum(is.na(temp))))
print(paste0("Sum of the number of american movies and missing values: ",sum(temp=="USA",na.rm=TRUE)+sum(is.na(temp))))
print(paste0("Sum of the number of american movies, foreign movies and missing values: ",
             sum(temp=="USA",na.rm=TRUE)+sum(is.na(temp))+sum(temp!="USA",na.rm=TRUE)))
print(paste0("Total number of observation: ",length(temp)))

[1] "Number of foreign movies: 216"
[1] "Number of american movies: 4559"
[1] "Number of missing values: 132"
[1] "Sum of the number of american movies and missing values: 4691"
[1] "Sum of the number of american movies, foreign movies and missing values: 4907"
[1] "Total number of observation: 4907"


Everything looks fine, so I copied the vector in the currency data.table. 

In [94]:
currency$currency<-temp

In [95]:
n<-c("title_year","currency","movie_title")
currency<-currency[,n,with=FALSE]

In [96]:
#write.csv(currency,"currency.csv",row.names=FALSE, col.names=TRUE)

# Merging the currency and the movie data

In [97]:
#currency<-fread("currency.csv", stringsAsFactors=FALSE)
#setDT(currency)

I found a data set of the historical exchange rate from 1950 to 2015 from the OECD website that I'll use to convert the budget of the movie in American dollars.

(https://data.oecd.org/conversion/exchange-rates.html OECD (2017), Exchange rates (indicator). doi: 10.1787/037ed317-en (Accessed on 13 January 2017)) 

In [98]:
ex_rate <- fread("ExRate50-15.csv",select=c("LOCATION","TIME","Value"),stringsAsFactors = F)
str(ex_rate)
print(paste0("Country code to change in currency: ",setdiff(unique(currency$currency),ex_rate$LOCATION)))

Classes 'data.table' and 'data.frame':	2764 obs. of  3 variables:
 $ LOCATION: chr  "AUS" "AUS" "AUS" "AUS" ...
 $ TIME    : int  1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 ...
 $ Value   : num  0.893 0.893 0.893 0.893 0.893 ...
 - attr(*, ".internal.selfref")=<externalptr> 
 [1] "Country code to change in currency: NA" 
 [2] "Country code to change in currency: FRF"
 [3] "Country code to change in currency: RUR"
 [4] "Country code to change in currency: CNY"
 [5] "Country code to change in currency: AUD"
 [6] "Country code to change in currency: HKD"
 [7] "Country code to change in currency: CAD"
 [8] "Country code to change in currency: JPY"
 [9] "Country code to change in currency: NOK"
[10] "Country code to change in currency: DEM"
[11] "Country code to change in currency: THB"
[12] "Country code to change in currency: KRW"
[13] "Country code to change in currency: HUF"
[14] "Country code to change in currency: INR"
[15] "Country code to change in currency: DKK"
[16] "Countr

This data set used the ISO 3166 country name abbreviations as an index for the table, while the IMDB website used the ISO 4217 currency codes to characterise the budget. In consequence, I'll have to map the abbreviations of the currency with the abbreviations of the country to be able to use these data. Lucky for me, I found a list of country code I can use for reference (http://data.okfn.org/data/core/country-codes).

In [99]:
abreviation <- fread("Abr.csv",stringsAsFactors = F)
#abreviation$Country<-substr(abreviation$Country, 1, nchar(abreviation$Country)-1)
str(abreviation)
setdiff(unique(currency$currency),abreviation$CODE)

Classes 'data.table' and 'data.frame':	235 obs. of  2 variables:
 $ CODE   : chr  "ABW" "AFG" "AFRI" "AGO" ...
 $ Country: chr  "Aruba" "Afghanistan" "Africa" "Angola" ...
 - attr(*, ".internal.selfref")=<externalptr> 


[1] NA     "EU28" "FRF"  "RUR"  "CNY"  "AUD"  "HKD"  "CAD"  "JPY"  "NOK" 
[11] "DEM"  "THB"  "KRW"  "HUF"  "INR"  "DKK"  "CZK"  "NZD"  "CHF"  "BRL" 
[21] "ZAR"  "SEK"

In [100]:
currency$currency[currency$currency=='FRF']<-"FRA"
currency$currency[currency$currency=='RUR']<-"USSR"
currency$currency[currency$currency=='CNY']<-"CHN"
currency$currency[currency$currency=='AUD']<-"AUS"
currency$currency[currency$currency=='HKD']<-"HKG"
currency$currency[currency$currency=='CAD']<-"CAN"
currency$currency[currency$currency=='JPY']<-"JPN"
currency$currency[currency$currency=='NOK']<-"NOR"
currency$currency[currency$currency=='DEM']<-"DEU"
currency$currency[currency$currency=='THB']<-"THA"
currency$currency[currency$currency=='KRW']<-"KOR"
currency$currency[currency$currency=='HUF']<-"HUN"
currency$currency[currency$currency=='INR']<-"IND"
currency$currency[currency$currency=='DKK']<-"DNK"
currency$currency[currency$currency=='CZK']<-"CZE"
currency$currency[currency$currency=='NZD']<-"NZL"
currency$currency[currency$currency=='CHF']<-"CHE"
currency$currency[currency$currency=='BRL']<-"BRA"
currency$currency[currency$currency=='ZAR']<-"ZAF"
currency$currency[currency$currency=='SEK']<-"SWE"

Let's see if there's currency code that I left out.

In [101]:
setdiff(unique(currency$currency),abreviation$CODE)

[1] NA     "EU28"

Since everything is ok, I can add the currency column, which I'll use in the next section to convert the budgets, to the data.table movies.

In [102]:
merge_movie<-full_join(movies,currency)
setDT(merge_movie)
head(merge_movie)

Joining, by = c("movie_title", "title_year")


color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genres_Reality.TV,genres_Romance,genres_Sci.Fi,genres_Short,genres_Sport,genres_Thriller,genres_War,genres_Western,movie_facebook_likes.1,currency
Color,James Cameron,723,178,0,855,Joel David Moore,1000,760505847,Action|Adventure|Fantasy|Sci-Fi,...,0,0,1,0,0,0,0,0,33000,NA
Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,309404152,Action|Adventure|Fantasy,...,0,0,0,0,0,0,0,0,0,NA
Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,200074175,Action|Adventure|Thriller,...,0,0,0,0,0,1,0,0,85000,NA
Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,448130642,Action|Thriller,...,0,0,0,0,0,1,0,0,164000,NA
Color,Doug Walker,NA,136,131,NA,Rob Walker,131,NA,Documentary,...,0,0,0,0,0,0,0,0,0,NA
Color,Andrew Stanton,462,132,475,530,Samantha Morton,640,73058679,Action|Adventure|Sci-Fi,...,0,0,1,0,0,0,0,0,24000,NA


In [103]:
merge_movie<-merge_movie[order(-budget)]

I make sure that all the labels in the variable "currency" are included in the data set from the OCED.

In [104]:
setdiff(unique(merge_movie$currency),ex_rate$LOCATION)

[1] "THA"  NA     "HKG"  "USSR"

Those values are not in the data set since these countries are not members of the OCED. Since there's not a lot of observations from Thailand or the USSR in the data set, I will enter those exchange rates manually in the table.

In [105]:
merge_movie[currency=='USSR']

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genres_Reality.TV,genres_Romance,genres_Sci.Fi,genres_Short,genres_Sport,genres_Thriller,genres_War,genres_Western,movie_facebook_likes.1,currency
Black and White,Andrei Tarkovsky,144,115,0,12,Anatoliy Solonitsyn,29,NA,Drama|Mystery|Sci-Fi,...,0,0,1,0,0,0,0,0,0,USSR


In [106]:
temp<-data.table(LOCATION="USSR",TIME=1972,Value=0.8290)
ex_rate<-rbind(ex_rate, temp,fill=TRUE)
tail(ex_rate)

LOCATION,TIME,Value
CRI,2011,505.6642
CRI,2012,502.9015
CRI,2013,499.7668
CRI,2014,538.3172
CRI,2015,534.5658
USSR,1972,0.8290


In [107]:
merge_movie[currency=='THA']

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genres_Reality.TV,genres_Romance,genres_Sci.Fi,genres_Short,genres_Sport,genres_Thriller,genres_War,genres_Western,movie_facebook_likes.1,currency
Color,Chatrichalerm Yukol,31,300,6,6,Chatchai Plengpanich,7,454255,Action|Adventure|Drama|History|War,...,0,0,0,0,0,0,1,0,124,THA
Color,Tony Jaa,110,110,0,7,Petchtai Wongkamlao,64,102055,Action,...,0,0,0,0,0,0,0,0,0,THA
Color,Prachya Pinkaew,112,111,64,380,Nathan Jones,778,11905519,Action|Crime|Drama|Thriller,...,0,0,0,0,0,1,0,0,0,THA


In [108]:
temp<-data.table(LOCATION=c('THA','THA','THA'),TIME=c(2001,2005,2008),Value=c(43.7900,39.6800,35.2024))
ex_rate<-rbind(ex_rate, temp,fill=TRUE)
tail(ex_rate)

LOCATION,TIME,Value
CRI,2014,538.3172
CRI,2015,534.5658
USSR,1972,0.8290
THA,2001,43.7900
THA,2005,39.6800
THA,2008,35.2024


In [109]:
merge_movie[currency=='HKG']

color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,genres_Reality.TV,genres_Romance,genres_Sci.Fi,genres_Short,genres_Sport,genres_Thriller,genres_War,genres_Western,movie_facebook_likes.1,currency
Color,Kaige Chen,90,103,45,107,Toby Leonard Moore,489,668171,Action|Drama|Fantasy,...,0,0,0,0,0,0,0,0,445,HKG
Color,Johnnie To,98,110,143,76,Anthony Chau-Sang Wong,155,49413,Action|Crime|Thriller,...,0,0,0,0,0,1,0,0,528,HKG


In [110]:
temp<-data.table(LOCATION=c('HKG','HKG'),TIME=c(2005,2006),Value=c(7.80,7.68))
ex_rate<-rbind(ex_rate, temp,fill=TRUE)
tail(ex_rate)

LOCATION,TIME,Value
USSR,1972,0.8290
THA,2001,43.7900
THA,2005,39.6800
THA,2008,35.2024
HKG,2005,7.8000
HKG,2006,7.6800


Also, I have created an entry in the exchange rate table with a "LOCATION" of NA, an exchange rate of 1 for the period from 1950 to 2015. By doing this, I didn't change the budget of the movie for which I don't know the country. Handling the missing values in the data set instead of handling those observations in my code will make calculation faster. 

In [111]:
temp<-data.table(LOCATION=rep(NA,times=66),TIME=1950:2015,Value=rep(1,times=66))
ex_rate<-rbind(ex_rate, temp,fill=TRUE)
tail(ex_rate)

LOCATION,TIME,Value
NA,2010,1
NA,2011,1
NA,2012,1
NA,2013,1
NA,2014,1
NA,2015,1


In [112]:
#write.csv(merge_movie,"movie_with_currency.csv",row.names=FALSE, col.names=TRUE)

# Convert budget

In [113]:
#merge_movie<-fread("movie_with_currency.csv", stringsAsFactors=FALSE)
#setDT(merge_movie)

Below, I have written a custom function who compares the country code and the year variables in the movies and ex_rate data sets to locate which exchange rate to apply on which movie budget. After that the function divides the budget by the exchange rate to get the budgets in U.S. dollars and return a vector of those values. 

In [114]:
#a=country of the rate
#b=year of the exchange rate
#c=value of the exchage rate
#x=country of the movie
#y=year of the making of the movie
#z=budget of the movie

convert_gross<-function(a,b,c,x,y,z){
    value<-z
    for (i in 1:length(y)){
       index<-which(a %in% x[i] & b %in%y [i])
        min<-1
        find<-FALSE
        if(length(index)==0)
        {
           value[i]<-z[i]
        }
        else
            {
            for (j in 1:length(index)){
            
            if(j==1)
                {
                    min<-c[index[j]]
                    find<-FALSE
                }           

            if(y[i]==b[index[j]])
            {
                value[i]<-z[i]/c[index[j]]
                Find<-TRUE
            }
        }
        
        if(!find)
            {
            
            value[i]<-z[i]/min
        }
            
        }
        
        
    } 
    return(value)
}

In [115]:
merge_movie$budget_us<-convert_gross(ex_rate$LOCATION,ex_rate$TIME,ex_rate$Value,
                                  merge_movie$currency,merge_movie$title_year,merge_movie$budget)

In [116]:
merge_movie<-merge_movie[!duplicated(merge_movie$movie_title)]

In [117]:
col<-c("movie_title","title_year","currency","budget","budget_us")
head(merge_movie[,col,with=FALSE])
head(merge_movie[is.na(merge_movie$currency),col,with=FALSE])

movie_title,title_year,currency,budget,budget_us
The Host,2006,KOR,12215500000,12793906
Lady Vengeance,2005,KOR,4200000000,4101095
Fateless,2005,HUN,2500000000,12526148
Princess Mononoke,1997,JPN,2400000000,19836209
Steamboy,2004,JPN,2127519898,19664196
Akira,1988,JPN,1100000000,8583579


movie_title,title_year,currency,budget,budget_us
Pirates of the Caribbean: At World's End,2007,NA,300000000,300000000
John Carter,2012,NA,263700000,263700000
Tangled,2010,NA,260000000,260000000
The Dark Knight Rises,2012,NA,250000000,250000000
Avengers: Age of Ultron,2015,NA,250000000,250000000
Harry Potter and the Half-Blood Prince,2009,NA,250000000,250000000


That table above show that the function handle correctly the movies with a budget in U.S. dollar or who have a missing value. Let's look at some European movies to see if the conversion was well handled. 

In [133]:
head(merge_movie[merge_movie$currency=="EU28",col,with=FALSE])

movie_title,title_year,currency,budget,budget_us
Asterix at the Olympic Games,2008,EU28,78000000,114256418
Two Brothers,2004,EU28,59660000,74078213
Oliver Twist,2005,EU28,50000000,62179774
Perfume: The Story of a Murderer,2006,EU28,50000000,62724160
Oceans,2009,EU28,40000000,55567672
Bandidas,2006,EU28,32000000,40143463


For exemple, the movie "Asterix at the Olympic Games" made in 2008 for 78 000 000 euro according to IMDB. So far, the table is good. That year, the exchage rate for the euro was 0.682675, so the budget should be, in U.S. dollar:

In [137]:
merge_movie[movie_title=="Asterix at the Olympic Games",budget]/ex_rate[LOCATION=="EU28"&TIME==2008,Value]

[1] 114256418

That seems alright! Let's look at the Asian films that skew the distribution.

In [120]:
head(merge_movie[merge_movie$currency=="KOR",col,with=FALSE])

movie_title,title_year,currency,budget,budget_us
The Host,2006,KOR,12215500000,12793906
Lady Vengeance,2005,KOR,4200000000,4101095


In [142]:
merge_movie[movie_title=="The Host",budget]/ex_rate[LOCATION=="KOR"&TIME==2006,Value]

[1] 12793906

The math checks out. Now, the only thing to look for is if there's still some outlier in the data.

In [122]:
merge_movie[merge_movie$budget_us>200000000,col,with=FALSE]

movie_title,title_year,currency,budget,budget_us
Tango,1998,ESP,700000000,779610729
The Messenger: The Story of Joan of Arc,1999,FRA,390000000,415500513
Pirates of the Caribbean: At World's End,2007,NA,300000000,300000000
John Carter,2012,NA,263700000,263700000
Tangled,2010,NA,260000000,260000000
Spider-Man 3,2007,USA,258000000,258000000
The Dark Knight Rises,2012,NA,250000000,250000000
Avengers: Age of Ultron,2015,NA,250000000,250000000
Harry Potter and the Half-Blood Prince,2009,NA,250000000,250000000
Batman v Superman: Dawn of Justice,2016,NA,250000000,250000000


In [123]:
merge_movie[merge_movie$budget_us>50000000&merge_movie$currency!="USA",col,with=FALSE]

movie_title,title_year,currency,budget,budget_us
Tango,1998,ESP,700000000,779610729
Red Cliff,2008,CHN,553632000,79674700
The Messenger: The Story of Joan of Arc,1999,FRA,390000000,415500513
Winged Migration,2001,FRA,160000000,143175453
Les couloirs du temps: Les visiteurs II,1998,FRA,140000000,155663655
Amen.,2002,FRA,103000000,96936432
The Widow of Saint-Pierre,2000,FRA,100000000,92131848
Asterix at the Olympic Games,2008,EU28,78000000,114256418
The Swindle,1997,FRA,60000000,67431035
Two Brothers,2004,EU28,59660000,74078213


After looking at the IMDB page of those movies, I realised that the budget of some European movie made between 1997 and 2002 were written in their native currency even if the euro was already introduced. For those years, the OCED listed the euro as the official currency of every country member of the European union even though the national currency were still in circulation. 

To estimate the budget of those movies in U.S. dollar I'll divide the budget by the exchange rate of the national currency in 1999.

In [156]:
merge_movie[!is.na(merge_movie$budget)&merge_movie$title_year>1997&merge_movie$currency=="FRA",col,with=FALSE]

movie_title,title_year,currency,budget,budget_us
The Messenger: The Story of Joan of Arc,1999,FRA,390000000,65000000
Winged Migration,2001,FRA,160000000,26666667
Les couloirs du temps: Les visiteurs II,1998,FRA,140000000,23333333
Amen.,2002,FRA,103000000,17166667
The Widow of Saint-Pierre,2000,FRA,100000000,16666667
Alias Betty,2001,FRA,50000000,44742329


In [125]:
merge_movie$budget_us[merge_movie$movie_title=="Amen."]<-merge_movie$budget[merge_movie$movie_title=="Amen."]/6
merge_movie$budget_us[merge_movie$movie_title=="Les couloirs du temps: Les visiteurs II"]<-
merge_movie$budget[merge_movie$movie_title=="Les couloirs du temps: Les visiteurs II"]/6
merge_movie$budget_us[merge_movie$movie_title=="The Messenger: The Story of Joan of Arc"]<-
merge_movie$budget[merge_movie$movie_title=="The Messenger: The Story of Joan of Arc"]/6
merge_movie$budget_us[merge_movie$movie_title=="The Swindle"]<-merge_movie$budget[merge_movie$movie_title=="The Swindle"]/6
merge_movie$budget_us[merge_movie$movie_title=="The Widow of Saint-Pierre"]<-
merge_movie$budget[merge_movie$movie_title=="The Widow of Saint-Pierre"]/6
merge_movie$budget_us[merge_movie$movie_title=="Winged Migration"]<-merge_movie$budget[merge_movie$movie_title=="Winged Migration"]/6

In [127]:
merge_movie$budget_us[merge_movie$movie_title=="Tango"]<-merge_movie$budget[merge_movie$movie_title=="Tango"]/152.0409
merge_movie[!is.na(merge_movie$budget_us)&merge_movie$budget>50000000&merge_movie$currency=="ESP",col,with=FALSE]

movie_title,title_year,currency,budget,budget_us
Tango,1998,ESP,700000000,4604024


Now that most of the outliers are dealt with, let's look again to the histogram of the "budget" variable. 

In [128]:
budget_hist <- plot_ly(x=~merge_movie$budget_us,type="histogram")
embed_notebook(budget_hist)

Warning message:
"Ignoring 632 observations"

In [129]:
write.csv(merge_movie,"movie_budget_us.csv",row.names=FALSE, col.names=TRUE)

Warning message in write.csv(merge_movie, "movie_budget_us.csv", row.names = FALSE, :
"attempt to set 'col.names' ignored"

# Conclusion

In this post I was able to reduce the number of missing data for the variables "gross" and "budget" by more than half. Also, I converted the value of the "budget" variable in a format that is accurate. In the next post, I'll explore the data by looking at the distribution of the variables and the relations between them.